In [1]:
import numpy as np
import tensorflow as tf
import spacy
import csv

from random import randint

nlp = spacy.load('en_core_web_sm')

In [2]:
# Load headlines

headlines = []
with open('test_100.csv', 'r+') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)
    for row in csv_reader:
        headlines.append(row[1])

In [3]:
# Tokenize and Lemmatize headlines

headlines_copy = list(headlines)
headlines = []
for headline in headlines_copy:
    headlines.append([ token.lemma_ for token in nlp(headline) ])

In [4]:
# Pad headlines

max_sequence_length = 26
numDimensions = 300
headlines_copy = list(headlines)
headlines = []
for headline in headlines_copy:    
    pad_array = np.zeros(max_sequence_length - len(headline)).tolist()
    new_headline = headline + pad_array
    headlines.append(new_headline)
    
print(headlines[2])

['sebi', "'s", 'omnibus', 'structure', 'move', 'boon', 'for', 'domestic', 'brokerage', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [5]:
# Load  pre-trained vectors

wordsList = np.load('wordsList.npy')
wordsList = wordsList.tolist()
wordsList = [ word.decode('UTF-8') for word in wordsList ]
wordVectors = np.load('wordVectors.npy')

print('Loaded embeddings.')

Loaded embeddings.


In [6]:
# Convert headlines to ids matrix.

id_matrix = []
for headline in headlines:
    id_headline = []
    for word in headline:
        try:
            id_headline.append(wordsList.index(word))
        except:
            id_headline.append(399999)
    id_matrix.append(id_headline)
            
print(id_matrix[2])

ids = id_matrix

[83616, 9, 28892, 1972, 483, 14758, 10, 1163, 6671, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999, 399999]


In [7]:
#rnn
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, max_sequence_length])
    for i in range(batchSize):
        if (i % 2 == 0): 
            num = randint(1,11499)
            labels.append([1,0])
        else:
            num = randint(13499,24999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, max_sequence_length])
    for i in range(batchSize):
        num = randint(11499,13499)
        if (num <= 12499):
            labels.append([1,0])
        else:
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels


tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, max_sequence_length])

data = tf.Variable(tf.zeros([batchSize, max_sequence_length, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

In [8]:
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
for i in range(iterations):
  #Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})

    #Write summary to Tensorboard
    if (i % 50 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)

    #Save the network every 10,000 training iterations
    if (i % 10000 == 0 and i != 0):
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)
writer.close()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



ValueError: cannot copy sequence with size 0 to array axis with dimension 26